# Bert NER on SageMaker using PyTorch

This uses the Biocreative II gene mention dataset https://biocreative.bioinformatics.udel.edu/tasks/biocreative-ii/task-1a-gene-mention-tagging/






In [1]:
import sys, os
import logging

sys.path.append("src")

logging.basicConfig(level="INFO", handlers=[logging.StreamHandler(sys.stdout)],
                        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

### Bucket and role set up

In [2]:
import sagemaker, boto3

from sagemaker import get_execution_role
sm_session = sagemaker.session.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')

# role=get_execution_role()
role ="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20181222T162635".format(account_id)


2020-10-03 15:31:49,761 - sagemaker.analytics - WARNING - pandas failed to import. Analytics features will be impaired or broken.
2020-10-03 15:31:50,013 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2020-10-03 15:31:50,091 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [3]:
data_bucket = "aegovan-data"

data_bucket_prefix = "Biocreative-gene-mention"

s3_uri_data = "s3://{}/{}/train data".format(data_bucket, data_bucket_prefix)
s3_uri_train = "{}/{}".format(s3_uri_data, "train.in")
s3_uri_classes = "{}/{}".format(s3_uri_data, "GENE.eval")

s3_uri_test = "s3://{}/{}/test data".format(data_bucket, data_bucket_prefix , "test.in")

s3_output_path = "s3://{}/{}/output".format(data_bucket, data_bucket_prefix)
s3_code_path = "s3://{}/{}/code".format(data_bucket, data_bucket_prefix)
s3_checkpoint = "s3://{}/{}/checkpoint3".format(data_bucket, data_bucket_prefix)

## Train

This shows you how to train BERT on SageMaker using SPOT instances

In [4]:
inputs_full =  {
    "train" : s3_uri_train,
    "class" : s3_uri_classes
}

inputs = inputs_full

In [5]:
sm_localcheckpoint_dir="/opt/ml/checkpoints/"

In [6]:
instance_type = "ml.p3.8xlarge"
instance_type_gpu_map = {"ml.p3.8xlarge":4, "ml.p3.2xlarge": 1, "ml.p3.16xlarge":8}

In [7]:
hp = {
"epochs" : 20,
"earlystoppingpatience" : 5,
# Increasing batch size might end up with CUDA OOM error, increase grad accumulation instead
"batch" : 8 * instance_type_gpu_map[instance_type],
"trainfile" :s3_uri_train.split("/")[-1],
"classfile":s3_uri_classes.split("/")[-1],
# The number of steps to accumulate gradients for
"gradaccumulation" : 4,
"log-level":"INFO",
# This param depends on your model max pos embedding size or when large you might end up with CUDA OOM error    
"maxseqlen" : 512,
# Make sure the lr is quite small, as this is a pretrained model..
"lr":0.00001,
# Use finetuning (set to 1), if you only want to change the weights in the final classification layer.. 
"finetune": 0,
"checkpointdir" : sm_localcheckpoint_dir,
# Checkpoints once every n epochs
"checkpointfreq": 2
}



In [8]:
hp

{'epochs': 20,
 'earlystoppingpatience': 5,
 'batch': 32,
 'trainfile': 'train.in',
 'classfile': 'GENE.eval',
 'gradaccumulation': 4,
 'log-level': 'INFO',
 'maxseqlen': 512,
 'lr': 1e-05,
 'finetune': 0,
 'checkpointdir': '/opt/ml/checkpoints/',
 'checkpointfreq': 2}

In [9]:
inputs

{'train': 's3://aegovan-data/Biocreative-gene-mention/train data/train.in',
 'class': 's3://aegovan-data/Biocreative-gene-mention/train data/GENE.eval'}

In [10]:
metric_definitions = [{"Name": "TrainLoss",
                     "Regex": "###score: train_loss### (\d*[.]?\d*)"}
                    ,{"Name": "ValidationLoss",
                     "Regex": "###score: val_loss### (\d*[.]?\d*)"}
                    ,{"Name": "TrainScore",
                     "Regex": "###score: train_score### (\d*[.]?\d*)"}
                   ,{"Name": "ValidationScore",
                     "Regex": "###score: val_score### (\d*[.]?\d*)"}
                    ]

In [11]:
# set True if you need spot instance
use_spot = True
train_max_run_secs =   2*24 * 60 * 60
spot_wait_sec =  5 * 60
max_wait_time_secs = train_max_run_secs +  spot_wait_sec

if not use_spot:
    max_wait_time_secs = None
    
# During local mode, no spot.., use smaller dataset
if instance_type == 'local':
    use_spot = False
    max_wait_time_secs = 0
    wait = True
    # Use smaller dataset to run locally
    inputs = inputs_sample


In [12]:
job_type = "bc2-ner-bert"
base_name = "{}".format(job_type)

In [16]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='main.py',
                    source_dir = 'src',
                    role=role,
                    framework_version ="1.4.0",
                    py_version='py3',
                    instance_count=1,
                    instance_type=instance_type,
                    hyperparameters = hp,
                    output_path=s3_output_path,
                    metric_definitions=metric_definitions,
                    volume_size=30,
                    code_location=s3_code_path,
                    debugger_hook_config=False,
                    base_job_name =base_name,  
                    use_spot_instances = use_spot,
                    max_run =  train_max_run_secs,
                    max_wait = max_wait_time_secs,   
                    checkpoint_s3_uri=s3_checkpoint,
                    checkpoint_local_path=sm_localcheckpoint_dir
                    )

estimator.fit(inputs, wait=True)

2020-10-03 16:46:02,171 - sagemaker - INFO - Creating training-job with name: bc2-ner-bert-2020-10-03-06-45-59-356
2020-10-03 06:46:03 Starting - Starting the training job...
2020-10-03 06:46:05 Starting - Launching requested ML instances......
2020-10-03 06:47:34 Starting - Preparing the instances for training......
2020-10-03 06:48:30 Downloading - Downloading input data...
2020-10-03 06:49:15 Training - Downloading the training image...
2020-10-03 06:50:05 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-10-03 06:50:06,305 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-10-03 06:50:06,347 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-10-03 06:50:06,351 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-10-03 06:50:06,666 sagemaker

2020-10-03 06:50:40,183 - trainer - INFO - Loading checkpoint /opt/ml/checkpoints/checkpoint.pt , found 1 checkpoint files
2020-10-03 06:50:44,469 - filelock - INFO - Lock 140322190054288 acquired on /root/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391.lock
2020-10-03 06:50:44,469 - transformers.file_utils - INFO - https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmppd0yycah
2020-10-03 06:50:44,543 - transformers.file_utils - INFO - storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json in cache at /root/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391
2020-10-03 06:50:44,544 - transformers.file_utils - INFO - creat

2020-10-03 06:51:35,648 - trainer - INFO - Train set result details:
2020-10-03 06:51:36,166 - trainer - INFO - Train set result details: 0.0774285206890428
2020-10-03 06:51:36,166 - trainer - INFO - Validation set result details:
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning,

2020-10-03 06:53:03,720 - trainer - INFO - Train set result details:
2020-10-03 06:53:04,254 - trainer - INFO - Train set result details: 0.3527971599951035
2020-10-03 06:53:04,254 - trainer - INFO - Validation set result details:
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning,

2020-10-03 06:54:32,876 - trainer - INFO - Train set result details:
2020-10-03 06:54:33,401 - trainer - INFO - Train set result details: 0.44400297988577103
2020-10-03 06:54:33,401 - trainer - INFO - Validation set result details:
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning

...

2020-10-03 07:14:02,079 - trainer - INFO - Train set result details:
2020-10-03 07:14:02,616 - trainer - INFO - Train set result details: 0.7214528351188921
2020-10-03 07:14:02,616 - trainer - INFO - Validation set result details:
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning,

2020-10-03 07:15:31,823 - trainer - INFO - Train set result details:
2020-10-03 07:15:32,562 - trainer - INFO - Train set result details: 0.7365409622886866
2020-10-03 07:15:32,562 - trainer - INFO - Validation set result details:
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning,

2020-10-03 07:17:01,266 - trainer - INFO - Train set result details:
2020-10-03 07:17:01,796 - trainer - INFO - Train set result details: 0.7345816420060234
2020-10-03 07:17:01,796 - trainer - INFO - Validation set result details:
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning,

2020-10-03 07:18:30,630 - trainer - INFO - Train set result details:
2020-10-03 07:18:31,158 - trainer - INFO - Train set result details: 0.7544456066945606
2020-10-03 07:18:31,158 - trainer - INFO - Validation set result details:
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning,

2020-10-03 07:20:04,184 - trainer - INFO - Train set result details:
2020-10-03 07:20:04,714 - trainer - INFO - Train set result details: 0.7625082508250826
2020-10-03 07:20:04,714 - trainer - INFO - Validation set result details:
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning,

## Deploy BERT model

#### Inference container
Ideally the server containing should already have all the required dependencies installed to reduce start up time and ensure that the runtime enviornment is consistent. This can be implemented using a custom docker image.

But for this demo, to simplify, we will let the Pytorch container script model install the dependencies during start up. As a result, you will see some of the initial ping requests fail, until all dependencies are installed.


In [14]:
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role
role = get_execution_role()

model_uri = estimator.model_data

model = PyTorchModel(model_data=model_uri,
                     role=role,
                     framework_version='1.4.0',
                     entry_point='serve.py',
                     source_dir='src')

predictor = model.deploy(initial_instance_count=1, instance_type='ml.p3.2xlarge')

2020-10-03 15:58:00,358 - sagemaker - WARNING - Couldn't call 'get_role' to get Role ARN from role name cfndeploy to get Role path.


ValueError: The current AWS identity is not a role: arn:aws:iam::324346001917:user/cfndeploy, therefore it cannot be used as a SageMaker execution role

### Invoke API

In [ ]:
data = ["Q-workshop is a Polish company located in Poznań that specializes in designand production of polyhedral dice",
        "ET is a sci-fi directed by steven spielberg"]

In [ ]:
from sagemaker.content_types import CONTENT_TYPE_CSV
import json

predictor.content_type = CONTENT_TYPE_CSV
predictor.serializer=None
predictor.deserializer = None


data_bytes="\n".join(data).encode("utf-8")
response_bytes  = predictor.predict(data_bytes,  
                                    initial_args={ "Accept":"text/json", "ContentType" : CONTENT_TYPE_CSV }
                                   )

json.loads(response_bytes.decode("utf-8"))

## Delete endpoint

In [ ]:
predictor.delete_endpoint()